- Required snowflake-ml-python version **1.5.0** or higher
- Required snowflake version **8.17** or higher
- Updated on: 5/5/2024

## Basic Feature Demo

This notebook demonstrates feature store with simple features. It includes an end-2-end ML experiment cycle: feature creation, training and inference. It also demonstrate the interoperation between Feature Store and Model Registry.

The notebooks also shows how to extract feature lineage and visualize the feature lineage graph.

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
import json

In [2]:
import json

with open('../../creds.json') as f:
    data = json.load(f)
    USERNAME = data['user']
    PASSWORD = data['password']
    SF_ACCOUNT = data['account']
    SF_WH = data['warehouse']

CONNECTION_PARAMETERS = {
   "account": SF_ACCOUNT,
   "user": USERNAME,
   "password": PASSWORD,
}

session = Session.builder.configs(CONNECTION_PARAMETERS).create()

In [3]:
from snowflake.snowpark.version import VERSION
snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION
from snowflake.ml import version
mlversion = version.VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
print('Snowflake ML version        : {}.{}.{}'.format(mlversion[0],mlversion[2],mlversion[4]))

User                        : RSHAH
Role                        : "RAJIV"
Database                    : "RAJIV"
Schema                      : "PUBLIC"
Warehouse                   : "RAJIV"
Snowflake version           : 8.26.0
Snowpark for Python version : 1.15.0a1
Snowflake ML version        : 1.5.0


## Setup Snowflake connection and database
For detailed session connection config, please follow this [tutorial](https://medium.com/snowflake/snowflakeloginoptions-an-easier-way-to-connect-using-python-2f0e726da936).


In [4]:
#session = Session.builder.configs(SnowflakeLoginOptions()).create()

Below cell creates temporary database, schema and warehouse for this notebook. All temporary resources will be cleaned up at the end of this notebook. You can rename with your own name if needed. 

In [5]:
# database name where test data, feature store and model lives.
FS_DEMO_DB = f"FEATURE_STORE_DEMO"
# schema where test data lives.
TEST_DATASET_SCHEMA = 'TEST_DATASET'
# feature store name.
FS_DEMO_SCHEMA = "AWESOME_FS_BASIC_FEATURES"
# the schema model lives.
MODEL_DEMO_SCHEMA = "MODELS"
# warehouse name used in this notebook.
FS_DEMO_WH = "RAJIV"

session.sql(f"CREATE OR REPLACE DATABASE {FS_DEMO_DB}").collect()
session.sql(f"""
    CREATE OR REPLACE SCHEMA {FS_DEMO_DB}.{TEST_DATASET_SCHEMA}
""").collect()
session.sql(f"""
    CREATE OR REPLACE SCHEMA {FS_DEMO_DB}.{MODEL_DEMO_SCHEMA}
""").collect()
session.sql(f"CREATE WAREHOUSE IF NOT EXISTS {FS_DEMO_WH}").collect()

[Row(status='RAJIV already exists, statement succeeded.')]

## Create a new FeatureStore client

Let's first create a feature store client. With `CREATE_IF_NOT_EXIST` mode, it will try to create schema and all necessary feature store metadata if it doesn't exist already. It is required for the first time to setup a Feature Store. Afterwards, you can use `FAIL_IF_NOT_EXIST` mode to connecte to an existing Feature Store. 

Note database must already exist. Feature Store will **NOT** try to create the database even in `CREATE_IF_NOT_EXIST` mode.

In [6]:
fs = FeatureStore(
    session=session, 
    database=FS_DEMO_DB, 
    name=FS_DEMO_SCHEMA, 
    default_warehouse=FS_DEMO_WH,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

## Prepare test data

We will use wine quality dataset for this demo. Download the public dataset from kaggle if you dont have it already: https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009. Replace `TEST_CSV_FILE_PATH` with your local file path.

In [7]:
TEST_CSV_FILE_PATH = 'winequality-red.csv'
session.file.put(
    f"file://{TEST_CSV_FILE_PATH}",session.get_session_stage())

from snowflake.snowpark.types import (
    StructType, 
    StructField, 
    IntegerType, 
    FloatType
)
input_schema = StructType(
    [
        StructField("fixed_acidity", FloatType()), 
        StructField("volatile_acidity", FloatType()), 
        StructField("citric_acid", FloatType()), 
        StructField("residual_sugar", FloatType()), 
        StructField("chlorides", FloatType()), 
        StructField("free_sulfur_dioxide", IntegerType()),
        StructField("total_sulfur_dioxide", IntegerType()), 
        StructField("density", FloatType()), 
        StructField("pH", FloatType()), 
        StructField("sulphates", FloatType()),
        StructField("alcohol", FloatType()), 
        StructField("quality", IntegerType())
    ]
)
df = session.read.options({"field_delimiter": ";", "skip_header": 1}) \
    .schema(input_schema) \
    .csv(f"{session.get_session_stage()}/winequality-red.csv")
full_table_name = f"{FS_DEMO_DB}.{TEST_DATASET_SCHEMA}.WINE_DATA"
df.write.mode("overwrite").save_as_table(full_table_name)

## Create and register a new Entity

We will create an Entity called *wine* and register it with the feature store.

You can retrieve the active Entities in the feature store with list_entities() API.

In [8]:
wine_entity = Entity(name="WINE", join_keys=["WINE_ID"])
fs.register_entity(wine_entity)
fs.list_entities().show()

-------------------------------------------
|"NAME"  |"JOIN_KEYS"  |"DESC"  |"OWNER"  |
-------------------------------------------
|WINE    |["WINE_ID"]  |        |RAJIV    |
-------------------------------------------



## Load source data and do some simple feature engineering

Then we will load from the source table and conduct some simple feature engineerings.

Here we are just doing two simple data manipulation (but more complex ones are carried out the same way):
1. Assign a WINE_ID column to the source
2. Derive a new column by multipying two existing feature columns

In [9]:
from snowflake.snowpark.window import Window

def addIdColumn(df, id_column_name):
    # Add id column to dataframe
    columns = df.columns
    new_df = df.withColumn(
        id_column_name,
        F.row_number().over(Window.order_by(F.col("quality"))))
    return new_df

source_df = session.table(full_table_name)
source_df = addIdColumn(source_df, "WINE_ID")

In [10]:
source_df_rows_count = source_df.count()
print(f"Total number of rows in source df: {source_df_rows_count}")
source_df.show()

Total number of rows in source df: 1599
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FIXED_ACIDITY"  |"VOLATILE_ACIDITY"  |"CITRIC_ACID"  |"RESIDUAL_SUGAR"  |"CHLORIDES"  |"FREE_SULFUR_DIOXIDE"  |"TOTAL_SULFUR_DIOXIDE"  |"DENSITY"  |"PH"  |"SULPHATES"  |"ALCOHOL"  |"QUALITY"  |"WINE_ID"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|11.6             |0.58                |0.66           |2.2               |0.074        |10                     |47                      |1.0008     |3.25  |0.57         |9.0        |3          |1          |
|8.3              |1.02                |0.02           |3.4               |0.084        |6                      |11             

In [11]:
def generate_new_feature(df):
    # Derive a new feature column
    new_df = df.withColumn(
        "MY_NEW_FEATURE", df["FIXED_ACIDITY"] * df["CITRIC_ACID"])
    return new_df.select([
        'WINE_ID',
        'FIXED_ACIDITY',
        'VOLATILE_ACIDITY',
        'CITRIC_ACID',
        'RESIDUAL_SUGAR',
        'CHLORIDES',
        'FREE_SULFUR_DIOXIDE',
        'TOTAL_SULFUR_DIOXIDE',
        'DENSITY',
        'PH',
        'MY_NEW_FEATURE',
    ])

feature_df = generate_new_feature(source_df)
feature_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"WINE_ID"  |"FIXED_ACIDITY"  |"VOLATILE_ACIDITY"  |"CITRIC_ACID"  |"RESIDUAL_SUGAR"  |"CHLORIDES"  |"FREE_SULFUR_DIOXIDE"  |"TOTAL_SULFUR_DIOXIDE"  |"DENSITY"  |"PH"  |"MY_NEW_FEATURE"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1          |11.6             |0.58                |0.66           |2.2               |0.074        |10                     |47                      |1.0008     |3.25  |7.656             |
|2          |8.3              |1.02                |0.02           |3.4               |0.084        |6                      |11                      |0.99892    |3.48  |0.166             |
|3          |7.4              |1.185               |0.0

## Create a new FeatureView and materialize the feature pipeline

Now we construct a Feature View with above DataFrame. We firstly create a draft feature view. We set the `refresh_freq` to 1 minute, so it will be refreshed every 1 minute. On the backend, it creates a Snowflake [dynamic table](https://docs.snowflake.com/en/user-guide/dynamic-tables-intro). At this point, the draft feature view will not take effect because it is not registered yet.
Then we register the feature view by via `register_feature_view`. It will materialize to Snowflake backend. [Incremental maintenance](https://docs.snowflake.com/en/user-guide/dynamic-tables-refresh#label-dynamic-tables-intro-refresh-queries) will start if the query is supported.

In [12]:
draft_fv = FeatureView(
    name="WINE_FEATURES", 
    entities=[wine_entity], 
    feature_df=feature_df, 
    refresh_freq="1 minute", 
    desc="my wine features auto refreshed on a schedule"
)
wine_features = fs.register_feature_view(
    feature_view=draft_fv, 
    version="1.0", 
    block=True
)

/Users/rajishah/anaconda3/envs/working38/lib/python3.8/site-packages/snowflake/ml/feature_store/feature_store.py:1066: UserWarning: Your pipeline won't be incrementally refreshed due to: "This dynamic table contains a complex query. Refresh mode has been set to FULL. If you wish to override this automatic choice, please re-create the dynamic table and specify REFRESH_MODE=INCREMENTAL. For best results, we recommend reading https://docs.snowflake.com/user-guide/dynamic-table-performance-guide before setting the refresh mode to INCREMENTAL.". It will likely incurr higher cost.
  self._check_dynamic_table_refresh_mode(feature_view_name)


We can examine the feature values in a feature view. 

In [13]:
fs.read_feature_view(wine_features).show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"WINE_ID"  |"FIXED_ACIDITY"  |"VOLATILE_ACIDITY"  |"CITRIC_ACID"  |"RESIDUAL_SUGAR"  |"CHLORIDES"  |"FREE_SULFUR_DIOXIDE"  |"TOTAL_SULFUR_DIOXIDE"  |"DENSITY"  |"PH"  |"MY_NEW_FEATURE"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1          |11.6             |0.58                |0.66           |2.2               |0.074        |10                     |47                      |1.0008     |3.25  |7.656             |
|2          |8.3              |1.02                |0.02           |3.4               |0.084        |6                      |11                      |0.99892    |3.48  |0.166             |
|3          |7.4              |1.185               |0.0

## Explore additional features

Now I have my FeatureView created with a collection of features, but what if I want to explore additional features on top?

Since a materialized FeatureView is immutable, we can create a new FeatureView for the additional features. Note `refresh_freq` of below Feature View is None. It means the Feature View is static and will not refresh on a schedule. You can still update the feature values by updating the data source (table `WINE_DATA`). On the backend it is a Snowflake view. 

In [14]:
extra_feature_df = source_df.select([
    'WINE_ID',
    'SULPHATES',
    'ALCOHOL',
])

extra_draft_fv = FeatureView(
    name="EXTRA_WINE_FEATURES", 
    entities=[wine_entity], 
    feature_df=extra_feature_df, 
    refresh_freq=None, 
    desc="extra wine features"
)
extra_features = fs.register_feature_view(
    feature_view=extra_draft_fv, 
    version="1.0", 
    block=True
)

We can examine the status of all feature views.

In [15]:
fs.list_feature_views(entity_name="WINE").show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"               |"VERSION"  |"DATABASE_NAME"     |"SCHEMA_NAME"              |"CREATED_ON"                |"OWNER"  |"DESC"                                         |"ENTITIES"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|EXTRA_WINE_FEATURES  |1.0        |FEATURE_STORE_DEMO  |AWESOME_FS_BASIC_FEATURES  |2024-07-17 11:26:36.532000  |RAJIV    |extra wine features                            |[           |
|                     |           |                    |                           |                            |         |                                               |  "WINE"    |
|                     |           |                    |                   

## Generate Training Data

After our feature pipelines are fully setup, we can start using them to generate training data and later do model prediction.

In [16]:
spine_df = source_df.select("WINE_ID", "QUALITY")
spine_df.show()

-------------------------
|"WINE_ID"  |"QUALITY"  |
-------------------------
|1          |3          |
|2          |3          |
|3          |3          |
|4          |3          |
|5          |3          |
|6          |3          |
|7          |3          |
|8          |3          |
|9          |3          |
|10         |3          |
-------------------------



Generate training data is easy since materialized FeatureViews already carry most of the metadata like join keys, timestamp for point-in-time lookup, etc. We just need to provide the spine data (it's called spine because we are essentially enriching the data by joining features with it). We can also generate dataset with a subset of features in the feature view by `slice`.

In [17]:
my_dataset = fs.generate_dataset(
    name="my_training_dataset",
    version="12",
    spine_df=spine_df, 
    features=[
        wine_features.slice([
            "FIXED_ACIDITY", "VOLATILE_ACIDITY", "CITRIC_ACID"]), 
        extra_features
    ],
    spine_timestamp_col=None, 
    spine_label_cols=["QUALITY"],
    exclude_columns=['WINE_ID'],
    desc="my training dataset with EXTRA_WINE_FEATURES and WINE_FEATURES",
)

Convert dataset to a snowpark dataframe and examine all the features in it.

In [18]:
training_data_df = my_dataset.read.to_snowpark_dataframe()
assert training_data_df.count() == source_df_rows_count
training_data_df.show()

-------------------------------------------------------------------------------------------------------------------------
|"QUALITY"  |"FIXED_ACIDITY"     |"VOLATILE_ACIDITY"  |"CITRIC_ACID"         |"SULPHATES"          |"ALCOHOL"           |
-------------------------------------------------------------------------------------------------------------------------
|3          |11.600000381469727  |0.5799999833106995  |0.6600000262260437    |0.5199999809265137   |10.199999809265137  |
|3          |8.300000190734863   |1.0199999809265137  |0.019999999552965164  |0.5400000214576721   |10.699999809265137  |
|3          |7.400000095367432   |1.184999942779541   |0.0                   |0.5699999928474426   |9.0                 |
|3          |6.800000190734863   |0.8149999976158142  |0.0                   |0.6299999952316284   |9.949999809265137   |
|3          |7.300000190734863   |0.9800000190734863  |0.05000000074505806   |0.49000000953674316  |11.0                |
|3          |10.39999961

## Train model with Snowpark ML

Now let's training a simple random forest model, and evaluate the prediction accuracy. When you call `fit()` on a DataFrame that converted from Feature Store Dataset, The linkage between model and dataset is automatically wired up. Later, you can easily retrieve the dataset from this model, or you can query the lineage about the dataset and model. This is work-in-progress and will be ready soon.

In [19]:
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling import metrics as snowml_metrics
from snowflake.snowpark.functions import abs as sp_abs, mean, col

def train_model_using_snowpark_ml(training_data_df):
    train, test = training_data_df.random_split([0.8, 0.2], seed=42)
    feature_columns = \
        [col for col in training_data_df.columns if col != "QUALITY"]
    label_column = "QUALITY"

    rf = RandomForestRegressor(
        input_cols=feature_columns, label_cols=[label_column], 
        max_depth=3, n_estimators=20, random_state=42
    )

    rf.fit(train)
    predictions = rf.predict(test)

    mse = snowml_metrics.mean_squared_error(
        df=predictions, 
        y_true_col_names=label_column, 
        y_pred_col_names="OUTPUT_" + label_column)

    accuracy = 100 - snowml_metrics.mean_absolute_percentage_error(
        df=predictions,
        y_true_col_names=label_column,
        y_pred_col_names="OUTPUT_" + label_column
    )

    print(f"MSE: {mse}, Accuracy: {accuracy}")
    return rf

rf = train_model_using_snowpark_ml(training_data_df)

The version of package 'snowflake-snowpark-python' in the local environment is 1.15.0a1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


MSE: 0.4325311509993623, Accuracy: 99.904377372923


## [Predict Optional 1] With local model
Now we can predict with a local model and the feature values retrieved from feature store. 

In [20]:
test_df = spine_df.limit(3).select("WINE_ID")

# load back feature views from dataset
fvs = fs.load_feature_views_from_dataset(my_dataset)
enriched_df = fs.retrieve_feature_values(test_df, fvs)
enriched_df = enriched_df.drop('WINE_ID')

In [21]:
pred = rf.predict(enriched_df.to_pandas())

print(pred)

   FIXED_ACIDITY  VOLATILE_ACIDITY  CITRIC_ACID  SULPHATES  ALCOHOL  \
0           11.6             0.580         0.66       0.52     10.2   
1            8.3             1.020         0.02       0.54     10.7   
2            7.4             1.185         0.00       0.57      9.0   

   OUTPUT_QUALITY  
0        5.247251  
1        5.223525  
2        5.148729  


## [Predict Option 2] With Model Registry
We can also predict with models in [Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops-model-registry).

### Step 1 : Log the model into Model Registry

Firstly, we connect to a model registry. 

In [22]:
from snowflake.ml.registry import Registry

registry = Registry(
    session=session, 
    database_name=FS_DEMO_DB, 
    schema_name=MODEL_DEMO_SCHEMA,
)

Then we log the model to model registry. Later, we can get it back with same model name and version.

In [23]:
model_name = "MY_RANDOM_FOREST_REGRESSOR_MODEL"

registry.log_model(
    model_name=model_name,
    version_name="V2",
    model=rf,
    comment="log my model trained with dataset",
)

/Users/rajishah/anaconda3/envs/working38/lib/python3.8/contextlib.py:113: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)


### Step 2 : Restore model and predict with features

We read the model back from model registry. We get the features from the dataset, and retrieve latest values for these features from Feature Store. We will same features that the model previously trained on for future inference.

We are working on retrieving dataset from a model directly. For now, we just use previously created dataset object. 

In [24]:
model = registry.get_model(model_name).version("V2")

# We are working on loading dataset back from a model. 
# For now, we use previously created dataset. 
fvs = fs.load_feature_views_from_dataset(my_dataset)
spine_df = spine_df.limit(3).select("WINE_ID")

enriched_df =fs.retrieve_feature_values(
    spine_df=spine_df, 
    features=fvs, 
    exclude_columns=["WINE_ID"]
)

Now we predict on the model and latest feature values.

In [25]:
restored_prediction = model.run(
    enriched_df.to_pandas(), function_name="predict")

print(restored_prediction)

   FIXED_ACIDITY  VOLATILE_ACIDITY  CITRIC_ACID  SULPHATES  ALCOHOL  \
0           11.6             0.580         0.66       0.52     10.2   
1            8.3             1.020         0.02       0.54     10.7   
2            7.4             1.185         0.00       0.57      9.0   

   OUTPUT_QUALITY  
0        5.247251  
1        5.223525  
2        5.148729  


## ML Lineage Demo
Shows output for a sample ML Lineage query.

In [30]:
TABLE_NAME = "WINE_DATA"
df = session.lineage.trace(f"{FS_DEMO_DB}.{TEST_DATASET_SCHEMA}.{TABLE_NAME}", "TABLE", direction="downstream", distance=4)

In [32]:
df.show()

--------------------------------------------------------------------------------------------------------------------------------------
|"SOURCE_OBJECT"                                     |"TARGET_OBJECT"                                     |"DIRECTION"  |"DISTANCE"  |
--------------------------------------------------------------------------------------------------------------------------------------
|{                                                   |{                                                   |Downstream   |1           |
|  "createdOn": "2024-07-17T18:26:23Z",              |  "createdOn": "2024-07-17T18:26:44Z",              |             |            |
|  "domain": "TABLE",                                |  "domain": "DATASET",                              |             |            |
|  "name": "FEATURE_STORE_DEMO.TEST_DATASET.WINE...  |  "name": "FEATURE_STORE_DEMO.AWESOME_FS_BASIC_...  |             |            |
|  "status": "ACTIVE"                                | 


 <img src="ml-lineage-example-viz.png" alt="ML Lineage Example" width="600"/>

To build your own ML viz of the lineage.

In [ ]:
import os
from graphviz import Digraph
from IPython.display import Image, display

def render_graph(df) -> None:
    df = df.to_pandas()
    graph = Digraph()
    
    # Track the nodes already added to avoid duplication
    added_nodes = set()
    
    for index, row in df.iterrows():    
        source_node = str(row['SOURCE_OBJECT']).replace(':', '-').replace('{', '').replace('}', '')
        target_node = str(row['TARGET_OBJECT']).replace(':', '-').replace('{', '').replace('}', '')
        
        if source_node not in added_nodes:
            graph.node(source_node)
            added_nodes.add(source_node)

        
        if target_node not in added_nodes:
            graph.node(target_node)
            added_nodes.add(target_node)
        
        graph.edge(source_node, target_node)
    
    graph.render('graph', format='png', cleanup=True)
    graph.view()

In [ ]:
#render_graph(df)

## Cleanup notebook
Cleanup resources created in this notebook.

In [26]:
#session.sql(f"DROP DATABASE IF EXISTS {FS_DEMO_DB}").collect()
#session.sql(f"DROP WAREHOUSE IF EXISTS {FS_DEMO_WH}").collect()